  # Introdução

Este notebook foi feito por Gabriel Matz visando o estudo dirigido para a prova de certificação oficial para o Tensorflow. A descrição da prova está no seguinte link:
https://www.tensorflow.org/certificate?hl=pt-br.

Utilizei como base os seguintes notebooks(além de sites variados e stack overflow para pequenas consultas):

1- https://github.com/mrdbourke/tensorflow-deep-learning

2-https://github.com/williamcwi/DeepLearning.AI-TensorFlow-Developer-Professional-Certificate

O conteúdo deste terceiro notebook engloba a quarta parte de exigências da prova do Tensorflow, que estão descritas abaixo:

"
(4) Processamento de Linguagem Natural (PLN)

Você precisa entender como usar redes neurais para resolver problemas de processamento de linguagem natural usando o TensorFlow. Você precisa saber como:

❏ Construir sistemas de processamento de linguagem natural usando TensorFlow.

❏ Preparar texto para usar em modelos TensorFlow.

❏ Construir modelos que identificam a categoria de um pedaço de texto usando categorização binária.

❏ Construir modelos que identificam a categoria de um pedaço de texto usando categorização multi-classe.

❏ Utilizar incorporações de palavras em seu modelo TensorFlow.

❏ Utilizar LSTMs em seu modelo para classificar texto para categorização binária ou multi-classe.

❏ Adicionar camadas RNN e GRU ao seu modelo.

❏ Utilizar RNNS, LSTMs, GRUs e CNNs em modelos que lidam com texto.

❏ Treinar LSTMs em texto existente para gerar texto (como músicas e poesias)."

# Desenvolvimento

**❏ Construir sistemas de processamento de linguagem natural usando TensorFlow.**

Para esse notebook será usado primeiramente o conjunto de dados de detecção de sarcasmo como visto em:

https://www.kaggle.com/code/pedrohft/nlp-tutorial-for-beginners-with-tensorflow/input

In [ ]:
import tensorflow as tf

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-884a60e2-ccee-ebba-bbd1-4396930d1363)


**❏ Preparar texto para usar em modelos TensorFlow.**

In [ ]:
import pandas as pd
datastore = pd.read_json('/content/Sarcasm_Headlines_Dataset.json.zip', lines = True)

sentences = datastore['headline'].tolist()
labels = datastore['is_sarcastic'].tolist()
urls = datastore['article_link'].tolist()

In [ ]:
datastore.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


Tirando dados inúteis

In [ ]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(datastore["headline"].to_numpy(),
                                                                            datastore["is_sarcastic"].to_numpy(),
                                                                            test_size=0.2,
                                                                            random_state=1)

In [ ]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(21367, 21367, 5342, 5342)

**❏ Utilizar incorporações de palavras(embeddings) em seu modelo TensorFlow.**

Passo 1: Tokenização

Transforma palavras em algo interpretável pela rede

In [ ]:
from tensorflow.keras.layers import TextVectorization
text_vectorizer = TextVectorization(max_tokens=10000, # palavras diferentes no texto
                                    standardize="lower_and_strip_punctuation", # método de normalização
                                    split="whitespace", # aonde separa os tokens
                                    ngrams=None, # grupo de n palavras
                                    output_mode="int", # tokens são inteiros
                                    output_sequence_length=15) # tamanho da seq de output
                                    # pad_to_max_tokens=True) # Se usar max_tokens faz o padding

In [ ]:
text_vectorizer.adapt(train_sentences)

In [ ]:
# Vizualizando resultado
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # mais comuns
bottom_5_words = words_in_vocab[-5:] # menos comuns
print(f"Quantidade de palavras: {len(words_in_vocab)}")
print(f"Top 5 mais comum: {top_5_words}")
print(f"5 menos comuns: {bottom_5_words}")

Quantidade de palavras: 10000
Top 5 mais comum: ['', '[UNK]', 'to', 'of', 'the']
5 menos comuns: ['scolded', 'scientology', 'sciencebacked', 'schoolhouse', 'schooler']


Passo 2: Camada de embedding

Camada a ser treinada que carrega o sentido entendido das palavras.

In [ ]:
embedding = tf.keras.layers.Embedding(input_dim=10000, # dimensão input
                                      output_dim=128, # dimensão de output
                                      embeddings_initializer="uniform", # Distribuição dos valores iniciais
                                      input_length=15, # tamanho input
                                      name="embedding_1")

embedding

**❏ Construir modelos que identificam a categoria de um pedaço de texto usando categorização binária.**

Abaixo veremos vários exemplos

**Baseline**

Sem muita coisa.

In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # input são strings
x = text_vectorizer(inputs) # vetoriza
x = embedding(x) # adiciona o embedding
x = layers.GlobalAveragePooling1D()(x) # diminui a dimensionalidade do embedding
outputs = layers.Dense(1, activation="sigmoid")(x) # output
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [ ]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
668/668 [==============================] - 14s 20ms/step - loss: 0.4924 - accuracy: 0.7765 - val_loss: 0.3727 - val_accuracy: 0.8458
Epoch 2/5
668/668 [==============================] - 4s 5ms/step - loss: 0.2971 - accuracy: 0.8826 - val_loss: 0.3316 - val_accuracy: 0.8585
Epoch 3/5
668/668 [==============================] - 3s 5ms/step - loss: 0.2305 - accuracy: 0.9111 - val_loss: 0.3281 - val_accuracy: 0.8622
Epoch 4/5
668/668 [==============================] - 5s 7ms/step - loss: 0.1884 - accuracy: 0.9300 - val_loss: 0.3486 - val_accuracy: 0.8557
Epoch 5/5
668/668 [==============================] - 3s 5ms/step - loss: 0.1595 - accuracy: 0.9414 - val_loss: 0.3791 - val_accuracy: 0.8525


**❏ Utilizar LSTMs em seu modelo para classificar texto para categorização binária ou multi-classe.**

In [ ]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_dense")

In [ ]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_2_history = model_2.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
668/668 [==============================] - 15s 20ms/step - loss: 0.1537 - accuracy: 0.9459 - val_loss: 0.4140 - val_accuracy: 0.8521
Epoch 2/5
668/668 [==============================] - 7s 10ms/step - loss: 0.1141 - accuracy: 0.9580 - val_loss: 0.4737 - val_accuracy: 0.8444
Epoch 3/5
668/668 [==============================] - 8s 12ms/step - loss: 0.0876 - accuracy: 0.9694 - val_loss: 0.5764 - val_accuracy: 0.8416
Epoch 4/5
668/668 [==============================] - 5s 8ms/step - loss: 0.0602 - accuracy: 0.9771 - val_loss: 0.5525 - val_accuracy: 0.8409
Epoch 5/5
668/668 [==============================] - 5s 7ms/step - loss: 0.0436 - accuracy: 0.9827 - val_loss: 0.8048 - val_accuracy: 0.8428


**❏ Adicionar camadas RNN e GRU ao seu modelo.**

In [ ]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_2_dense")

In [ ]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_3_history = model_3.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

Epoch 1/5
668/668 [==============================] - 15s 19ms/step - loss: 0.1038 - accuracy: 0.9599 - val_loss: 0.6885 - val_accuracy: 0.8396
Epoch 2/5
668/668 [==============================] - 5s 8ms/step - loss: 0.0432 - accuracy: 0.9849 - val_loss: 0.7374 - val_accuracy: 0.8411
Epoch 3/5
339/668 [==============>...............] - ETA: 1s - loss: 0.0305 - accuracy: 0.9888

KeyboardInterrupt: 

**❏ Utilizar RNNS, LSTMs, GRUs e CNNs em modelos que lidam com texto.**


**Bidirecional**

In [ ]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_2_dense")

In [ ]:
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_4_history = model_4.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

**CNN**

In [ ]:
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_dense")

In [ ]:
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_5_history = model_5.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels))

**❏ Construir modelos que identificam a categoria de um pedaço de texto usando categorização multi-classe.**

Baseado no problema:

https://www.kaggle.com/code/john77eipe/multi-class-text-classification-with-scikit-learn/notebook

Temos que classificar a categoria da reclamação baseada no texto.

In [ ]:
import pandas as pd
import csv
df = pd.read_csv('/content/rows.csv', header = 0, quoting=csv.QUOTE_NONE, encoding='utf-8', on_bad_lines='skip')
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,05/10/2019,Checking or savings account,Checking account,Managing an account,Problem using a debit or ATM card,NaN,NaN,NAVY FEDERAL CREDIT UNION,FL,328XX,Older American,NaN,Web,05/10/2019,In progress,Yes,NaN,3238275
1,05/10/2019,Checking or savings account,Other banking product or service,Managing an account,Deposits and withdrawals,NaN,NaN,BOEING EMPLOYEES CREDIT UNION,WA,98204,NaN,NaN,Referral,05/10/2019,Closed with explanation,Yes,NaN,3238228
2,05/10/2019,Debt collection,Payday loan debt,Communication tactics,Frequent or repeated calls,NaN,NaN,CURO Intermediate Holdings,TX,751XX,NaN,NaN,Web,05/10/2019,Closed with explanation,Yes,NaN,3237964
3,05/10/2019,Checking or savings account,Checking account,Managing an account,Banking errors,NaN,NaN,ALLY FINANCIAL INC.,AZ,85205,NaN,NaN,Postal mail,05/10/2019,In progress,Yes,NaN,3238460
4,05/10/2019,Mortgage,Other type of mortgage,Closing on a mortgage,NaN,NaN,NaN,Statebridge Company,NJ,08302,NaN,NaN,Web,05/10/2019,Closed with explanation,Yes,NaN,3237885


In [ ]:
df.size

623394

In [ ]:
col = ['Product', 'Consumer complaint narrative']
df = df[col].dropna()
df.head()

,Product,Consumer complaint narrative
8806,"""Credit reporting",Incorrect information on your report
8808,"""Credit reporting",Incorrect information on your report
8809,Company has responded to the consumer and the ...,292XX
8864,Debt collection,i never received anything stating that i owed ...
8891,"""Credit reporting",Incorrect information on your report


In [ ]:
df.size

15980

In [ ]:
from sklearn.model_selection import train_test_split
train, test= train_test_split(df, test_size=0.20, shuffle=True)
test.head(), train.head()

(                                                 Product  \
 31722                        Credit card or prepaid card   
 25188   but their voice recorder says they will not t...   
 13795                                           Mortgage   
 19455                        Credit card or prepaid card   
 23376                                 however inaccurate   
 
                             Consumer complaint narrative  
 31722  "Membership Fee issue with Citibank Membership...  
 25188   just refuse the shipment when it comes. I ALS...  
 13795  "I got my home in XXXX with a 30 year mortgage...  
 19455  "An unauthorized Amex card was taken in my nam...  
 23376                                               FSB"  ,
                                                  Product  \
 29515         and provided a copy of our payment receipt   
 29040  Company has responded to the consumer and the ...   
 33383   I received an email from Equifax with a confi...   
 26665                     

In [ ]:
train.size, test.size

(12784, 3196)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse_output=False)
train_labels_one_hot = one_hot_encoder.fit_transform(train["Product"].to_numpy().reshape(-1, 1))
test_labels_one_hot = one_hot_encoder.transform(test["Product"].to_numpy().reshape(-1, 1))

ValueError: Found unknown categories [' I realized that my XXXX-brand washing machine was no longer draining properly', ' Mr XXXX', ' XXXX XXXX and XXXX XXXX XXXX both continue to report these accounts negatively. The negative reporting is driving my credit score down and I need the information to be updated with the CRAs to reflect that these disputes have been settled and paid in full."', ' but that he would send it to the dealership directly so they could contact me. No one from the dealership ever reached out for resolution."', ' you charge a significant fee for anyone to make a payment online or by phone', ' Georgia law ( Official Code of Georgia', ' this shipment came in TWO packages on different days. I was not home so XXXX dropped both shipments off at my doorstep ( 2 consecutive days ). I never opened them', ' then cancelled the ( 2 ) online "" classes. \'\' There were NO "" classes \'\' it was a total scam. There was NOTHING of value provided to me at all or in anyway', ' whi

In [ ]:
train_sentences = train["Consumer complaint narrative"].tolist()
test_sentences = test["Consumer complaint narrative"].tolist()

In [ ]:
from tensorflow.keras.layers import TextVectorization

# Before TensorFlow 2.6
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens=50000, # number of words in vocabulary
                                    output_sequence_length=55) # desired output length of vectorized sequences

In [ ]:
# Create token embedding layer
import tensorflow as tf
token_embed = tf.keras.layers.Embedding(input_dim=20000, # length of vocabulary
                               output_dim=128, # Note: different embedding sizes result in drastically different numbers of parameters to train
                               # Use masking to handle variable sequence lengths (save space)
                               mask_zero=True,
                               name="token_embedding")

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_sentences, train_labels_one_hot))
test_dataset = tf.data.Dataset.from_tensor_slices((test_sentences, test_labels_one_hot))

In [ ]:
# Create 1D convolutional model to process sequences
inputs = layers.Input(shape=(1,), dtype=tf.string)
text_vectors = text_vectorizer(inputs) # vectorize text inputs
token_embeddings = token_embed(text_vectors) # create embedding
x = layers.Conv1D(64, kernel_size=5, padding="same", activation="relu")(token_embeddings)
x = layers.GlobalAveragePooling1D()(x) # condense the output of our feature vector
outputs = layers.Dense(num_classes, activation="softmax")(x)
model_1 = tf.keras.Model(inputs, outputs)

# Compile
model_1.compile(loss="categorical_crossentropy", # if your labels are integer form (not one hot) use sparse_categorical_crossentropy
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_1_history = model_1.fit(train_dataset,
                              steps_per_epoch=int(0.1 * len(train_dataset)), # only fit on 10% of batches for faster training time
                              epochs=3,
                              validation_data=test_dataset,
                              validation_steps=int(0.1 * len(test_dataset))) # only validate on 10% of batches

**❏ Treinar LSTMs em texto existente para gerar texto (como músicas e poesias)**

https://www.tensorflow.org/text/tutorials/text_generation?hl=pt-br